In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)

import os
sys.path.append(os.path.join(os.environ['VIRTUAL_ENV'],f"lib/python{'.'.join([str(x) for x in sys.version_info[0:2]])}/site-packages")) 
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
import json

from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tsfresh.feature_extraction import settings
from sklearn.model_selection import train_test_split

from data_pipeline import DataPipeline
from vae import VAE

2023-06-23 12:59:36,045 INFO    Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [3]:
repeat_nums = [0,1,2] #You can give a list of integers between 0 - 10
expConfig_nums = [0,1,2,3,4,5]

In [4]:
#You can either use one of the feature extraction methods or use the features determined beforehand
use_pre_selected_features = True
extracted_data = False

In [5]:
data_dir = f"/projectnb/peaclab-mon/aksar/deployment_experiments/eclipse/new_dataset_experiments/paper_dataset"
selected_features_filename = "/projectnb/peaclab-mon/aksar/deployment_experiments/eclipse/synthetic_dataset_experiments/fe_eclipse_tsfresh_raw_CHI_2000.json"

In [6]:
# "paper_vae_best" -> this one has all applications and healthy and memleak runs.
model_folder_output_name = "ae_experiments_vae"
output_dir = f"/projectnb/peaclab-mon/aksar/deployment_experiments/eclipse/sc_ae_experiments/models/{model_folder_output_name}"

In [7]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print("Created outputs directory")   
else:
    print("Output directory already exists")
    
if not os.path.exists(output_dir + "/results"):
    os.makedirs(output_dir + "/results")
    print("Created results directory")   
else:
    print("Results directory already exists")       

Output directory already exists
Results directory already exists


In [8]:
for repeat_num in repeat_nums:
    
    print(f"######### Repeat Num: {repeat_num}  #########")
    
    for expConfig_num in expConfig_nums:
        
        print(f"######### Experimental Configuration: {expConfig_num}  #########")
                        
        if use_pre_selected_features: 
            with open(selected_features_filename, "r") as fp:
                selected_features_json = json.load(fp)

            fe_selected_features = selected_features_json['tsfresh_column_names']
                        
        if expConfig_num == 0:
            healthy_test_data_percentage = 0.2
        elif expConfig_num == 1:
            healthy_test_data_percentage = 0.4
        elif expConfig_num == 2:
            healthy_test_data_percentage = 0.6
        elif expConfig_num == 3:
            healthy_test_data_percentage = 0.8
        elif expConfig_num == 4:
            healthy_test_data_percentage = 0.9        
        elif expConfig_num == 5:
            healthy_test_data_percentage = 0.95                    
            
            
        if not os.path.exists(output_dir):
            logging.info(f"Creating: {output_dir}")
            os.makedirs(output_dir)            
            
            
        pipeline = DataPipeline()
        x_train, y_train, x_test, y_test, x_val, y_val = pipeline.load_HPC_data(data_dir)
        
        
        all_y = pd.concat([y_train, y_test])
        print(all_y.shape)

        all_x = pd.concat([x_train, x_test])
        all_x.set_index(['job_id', 'component_id'],inplace=True)
        print(all_x.shape)        
        
        
        selected_apps = ['exa', 'lammps', 'sw4', 'sw4lite']
        #selected_apps = ['exa', 'lammps']
        selected_labels = ['none', 'memleak']
        
        
        exp_config_dict = {
            'selected_apps': selected_apps,
            'selected_labels': selected_labels,
            'dataset_stats' : {
                                'train': {
                                            '0': 0,
                                            '1': 0
                                         },
                                'test': {
                                            '0': 0,
                                            '1': 0                        
                                }
                                }
        }        
        
        
        curr_all_y = all_y[(all_y['app_name'].isin(selected_apps)) & (all_y['anom_name'].isin(selected_labels))]        
        
        
        healthy_labels = curr_all_y[curr_all_y['binary_anom'] == 0]
        anom_labels = curr_all_y[curr_all_y['binary_anom'] != 0]
        
        #Train test split on the healthy node_ids
        train_label_healthy, test_label_healthy = train_test_split(healthy_labels, 
                                                                   test_size=healthy_test_data_percentage)#, random_state=0)

        print(train_label_healthy.shape)
        print(test_label_healthy.shape)

        exp_config_dict['dataset_stats']['train']['0'] = train_label_healthy.shape[0]
        exp_config_dict['dataset_stats']['test']['0'] = test_label_healthy.shape[0]       

        #Train data only have healthy node_ids
        x_train = all_x.loc[train_label_healthy.index]
        y_train = train_label_healthy.copy()

        assert set(x_train.index.get_level_values('component_id')) == set(y_train.index.get_level_values('component_id'))

        print(f"Train data shape: {x_train.shape} with {len(set(x_train.index.get_level_values('component_id')))} unique jobid compid combos")
        print(f"Train label dist\n{y_train['binary_anom'].value_counts()}")

        assert len(x_train.index.unique()) == len(y_train)   
        
        #Test data will have some healthy and anomalous node_ids
        test_data_healthy = all_x.loc[test_label_healthy.index]
        assert set(test_data_healthy.index.get_level_values('component_id')) == set(test_label_healthy.index.get_level_values('component_id'))

        anom_data = all_x.loc[anom_labels.index]
        exp_config_dict['dataset_stats']['test']['1'] = anom_labels.shape[0]

        assert set(anom_data.index.get_level_values('component_id')) == set(anom_labels.index.get_level_values('component_id'))

        x_test = pd.concat([test_data_healthy, anom_data])
        y_test = pd.concat([test_label_healthy, anom_labels])

        assert set(x_test.index.get_level_values('component_id')) == set(y_test.index.get_level_values('component_id'))

        print(f"Test data shape: {x_test.shape}")
        print(f"Test label dist\n{y_test['binary_anom'].value_counts()}")


        assert len(x_test.index.unique()) == len(y_test)      
        
        x_train.reset_index(inplace=True)
        x_test.reset_index(inplace=True)
        
        #Generate features for the train data
        if use_pre_selected_features:
            x_train_fe = pipeline.tsfresh_generate_features(x_train, fe_config=None, kind_to_fc_parameters=fe_selected_features)
        elif not extracted_data:
            x_train_fe = pipeline.tsfresh_generate_features(x_train, fe_config="minimal")
        else:
            x_train_fe = x_train.copy(deep=True)
            logging.info(f"Features are already extracted")

        if not (y_train is None):
            y_train = y_train.loc[x_train_fe.index]        

        assert all(y_train.index == x_train_fe.index)            
        
        
        #Generate features for the test data
        if not (x_test) is None: 
            if use_pre_selected_features:
                x_test_fe = pipeline.tsfresh_generate_features(x_test, fe_config=None, kind_to_fc_parameters=fe_selected_features)    
            elif not (extracted_data):
                x_test_fe = pipeline.tsfresh_generate_features(x_test, fe_config="minimal")
            else:
                x_test_fe = x_test.copy(deep=True)
                logging.info(f"Features are already extracted")

        if not (y_test is None):
            y_test = y_test.loc[x_test_fe.index]

        assert all(y_test.index == x_test_fe.index)            
        
        
        #Make the number of columns and the order equal
        if not (x_test) is None:

            if len(x_test_fe.columns) < len(x_train_fe.columns):
                x_train_fe = x_train_fe[x_test_fe.columns]
                print(x_train_fe.shape)

            elif len(x_test_fe.columns) > len(x_train_fe.columns):
                x_test_fe = x_test_fe[x_train_fe.columns]
                print(x_test_fe.shape)

            x_train_fe = x_train_fe[x_test_fe.columns]    
            assert all(x_train_fe.columns == x_test_fe.columns)    
            x_test_fe = x_test_fe.loc[y_test.index]


        #Generate features for the val data
        if not (x_val) is None: 
            if not (extracted_data):
                x_val_fe = pipeline.tsfresh_generate_features(x_val, fe_config="minimal")
            else:
                x_val_fe = x_val.copy(deep=True)
                logging.info(f"Features are already extracted")

        if not (y_val is None):
            y_val = y_val.loc[x_val_fe.index]    


        #Apply scaler to train and test data(if it exists)
        if not (x_test is None):
            x_train_scaled, x_test_scaled = pipeline.scale_data(x_train_fe, x_test_fe, save_dir=output_dir)
        else:
            x_train_scaled, x_test_scaled = pipeline.scale_data(x_train_fe, None, save_dir=output_dir)

        logging.info(f"X_train scaled shape: {x_train_scaled.shape}")      
            
            
        input_dim = x_train_scaled.shape[1]
        intermediate_dim = int(input_dim / 2)
        latent_dim = int(input_dim / 3)
        
        
        if 'vae' in locals():
            print("Vae exists; will delete to be safe")
            del vae
        else:
            print("Vae is not defined")

        vae = VAE(
                    name="model",
                    input_dim=input_dim,
                    intermediate_dim=intermediate_dim,
                    latent_dim=latent_dim,
                    learning_rate=1e-4
        )
        
        
        # train the VAE model
        vae.fit(
                x_train=x_train_scaled,
                epochs=1000,
                batch_size=32, 
                validation_split=0.1,
                save_dir=output_dir,
                verbose=0
        )        
        
        deployment_metadata = {
            'threshold': vae.threshold,
            #Column order is important when feeding the data
            'raw_column_names': list(x_train_scaled.columns),
            'fe_column_names': settings.from_columns(list(x_train_scaled.columns))
        }

        with open(Path(output_dir) / 'deployment_metadata.json', 'w') as fp:
            json.dump(deployment_metadata, fp)
            
            
        y_pred_train, x_train_recon_errors = vae.predict_anomaly(x_train_scaled)

        if not (y_train is None):
            print("Classification Report in Training Data\n")
            print(classification_report(y_train['binary_anom'].values, y_pred_train))        
            
            
        if not (x_test is None):

            assert all(x_test_scaled.columns == x_train_scaled.columns)
            assert all(x_test_scaled.index == y_test.index)

            y_pred_test, x_test_recon_errors = vae.predict_anomaly(x_test_scaled)    
            logging.info(f"Test data prediction results: {y_pred_test}")
            logging.info(f"Selected threshold value\n: {vae.threshold}")
            #logging.info(f"Test data reconstruction errors\n: {x_test_recon_errors}")          
            
        #If the labeled test data exists, generate results
        if not (y_test is None):
            y_test['binary_pred'] = y_pred_test
            y_test['recon_errors'] = x_test_recon_errors
            print("Classification Report in Test Data\n")
            print(classification_report(y_test['binary_anom'].values, y_pred_test))  

            result_dict = classification_report(y_test['binary_anom'].values, y_pred_test, output_dict= True)                        

            
        # Writing to sample.json     
        with open( Path(output_dir) / "results" / f"expConfig_{expConfig_num}_repeatNum_{repeat_num}_testResults.json", "w") as outfile:
            json.dump(result_dict, outfile)

        # Writing to sample.json
        with open( Path(output_dir) / "results" / f"expConfig_{expConfig_num}_repeatNum_{repeat_num}_dataStats.json", "w") as outfile:
            json.dump(exp_config_dict, outfile)                
            
            
        print(f"################################################################################################################################################")            

######### Repeat Num: 0  #########
######### Experimental Configuration: 0  #########


2023-06-23 12:59:41,246 ERROR   File not found!: /projectnb/peaclab-mon/aksar/deployment_experiments/eclipse/new_dataset_experiments/paper_dataset/prod_val_data.hdf
2023-06-23 12:59:41,247 ERROR   File not found!: /projectnb/peaclab-mon/aksar/deployment_experiments/eclipse/new_dataset_experiments/paper_dataset/prod_val_label.csv
2023-06-23 12:59:43,667 INFO    Data read successfully
2023-06-23 12:59:43,668 INFO    Shape of x_train: (145826, 159)
2023-06-23 12:59:43,669 INFO    Shape of y_train: (64, 4)
2023-06-23 12:59:43,669 INFO    Shape of x_test: (375239, 159)
2023-06-23 12:59:43,670 INFO    Shape of y_test: (176, 4)


(240, 4)
(521065, 157)
(64, 4)
(16, 4)
Train data shape: (138230, 157) with 57 unique jobid compid combos
Train label dist
0    64
Name: binary_anom, dtype: int64
Test data shape: (210147, 157)
Test label dist
1    80
0    16
Name: binary_anom, dtype: int64


2023-06-23 12:59:46,207 INFO    TSFRESH will use kind_to_fc_parameters
Feature Extraction: 100%|██████████| 40/40 [00:43<00:00,  1.09s/it]
2023-06-23 13:00:30,668 INFO    Feature extraction: Before dropping NaNs: (64, 2000)
2023-06-23 13:00:30,678 INFO    Feature extraction: Dropped NaNs: (64, 2000)
2023-06-23 13:00:31,139 INFO    TSFRESH will use kind_to_fc_parameters
Feature Extraction: 100%|██████████| 40/40 [01:24<00:00,  2.11s/it]
2023-06-23 13:01:56,257 INFO    Feature extraction: Before dropping NaNs: (96, 2000)
2023-06-23 13:01:56,269 INFO    Feature extraction: Dropped NaNs: (96, 2000)
2023-06-23 13:01:56,363 INFO    x_test is not None, scaling
2023-06-23 13:01:56,395 INFO    Scaler is saved
2023-06-23 13:01:56,395 INFO    X_train scaled shape: (64, 2000)
2023-06-23 13:01:56,587 WARNING OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.


Vae is not defined


...

(240, 4)
(521065, 157)
(4, 4)
(76, 4)
Train data shape: (11391, 157) with 4 unique jobid compid combos
Train label dist
0    4
Name: binary_anom, dtype: int64
Test data shape: (336986, 157)
Test label dist
1    80
0    76
Name: binary_anom, dtype: int64


2023-06-23 13:45:33,375 INFO    TSFRESH will use kind_to_fc_parameters
Feature Extraction: 100%|██████████| 39/39 [00:04<00:00,  8.95it/s]
2023-06-23 13:45:38,743 INFO    Feature extraction: Before dropping NaNs: (4, 2000)
2023-06-23 13:45:38,754 INFO    Feature extraction: Dropped NaNs: (4, 2000)
2023-06-23 13:45:39,330 INFO    TSFRESH will use kind_to_fc_parameters
Feature Extraction: 100%|██████████| 40/40 [01:59<00:00,  2.99s/it]
2023-06-23 13:47:40,443 INFO    Feature extraction: Before dropping NaNs: (156, 2000)
2023-06-23 13:47:40,455 INFO    Feature extraction: Dropped NaNs: (156, 2000)
2023-06-23 13:47:40,554 INFO    x_test is not None, scaling
2023-06-23 13:47:40,588 INFO    Scaler is saved
2023-06-23 13:47:40,589 INFO    X_train scaled shape: (4, 2000)


Vae exists; will delete to be safe


2023-06-23 13:47:42,399 WARNING Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
/project/peaclab-mon/comte_py36_notf/lib/python3.6/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/project/peaclab-mon/comte_py36_notf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/project/peaclab-mon/comte_py36_notf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall a

Classification Report in Training Data

              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.00      0.00      0.00         0

    accuracy                           0.75         4
   macro avg       0.50      0.38      0.43         4
weighted avg       1.00      0.75      0.86         4

Classification Report in Test Data

              precision    recall  f1-score   support

           0       0.48      0.99      0.65        76
           1       0.00      0.00      0.00        80

    accuracy                           0.48       156
   macro avg       0.24      0.49      0.32       156
weighted avg       0.24      0.48      0.32       156

################################################################################################################################################
